# Análise de Infraestrutura de Domicílios

Vamos começar obtendo os dados de interesse (Fonte IBGE 2010):

- 0  Domicílios particulares e domicílios coletivos
- 11 Domicílios particulares permanentes com abastecimento de água da rede geral
- 15 Domicílios particulares permanentes com banheiro de uso exclusivo dos moradores ou sanitário
- 16 Domicílios particulares permanentes com banheiro de uso exclusivo dos moradores ou sanitário e esgotamento sanitário via rede geral de esgoto ou pluvial
- 22 Domicílios particulares permanentes sem banheiro de uso exclusivo dos moradores e nem sanitário
- 23 Domicílios particulares permanentes com banheiro de uso exclusivo dos moradores
- 33 Domicílios particulares permanentes sem banheiro de uso exclusivo dos moradores
- 35 Domicílios particulares permanentes com lixo coletado por serviço de limpeza
- 43 Domicílios particulares permanentes com energia elétrica de companhia distribuidora


In [1]:
import os
import sys

#path para a biblioteca do apiModulo. Ajuste de acordo com a necessidade
sys.path.insert(0, os.path.abspath('../../..'))
from apiModulo.api import *

## Pré-requisitos: 
- Usaremos o campo "Domicílios particulares e domicílios coletivos" para normalizar todos os demais valores e analisar de acordo com a proporção de domicílio.
- Vamos filtrar as geometrias para a área de estudo do Anél Viário

In [2]:
#obtém a região de análise do centro de São Luís
centro = obterCamada('anel_viario', simples=True)

total, m = obterIndicador(0)
total = limparDados(total) #limpeza de valores não definidos
total = total.rename(columns={"ibge_domicilio01_v001":"total"})
total = juncaoEspacial(total, centro, tipo='inner', predicado='intersects')
total = total.drop(columns=['geometria'])

## Infraestrutura em termos de abastecimento de água:
- 11 Domicílios particulares permanentes com abastecimento de água da rede geral

In [3]:
import pprint

agua, m = obterIndicador(11)
pprint.pprint(m) #imprime as variáveis presentes na informação

#obtendo dados apenas para a área de estudo
agua = juncaoEspacial(agua, centro, tipo='inner', predicado='intersects')
agua = limparDados(agua) #limpeza de valores não definidos

agua = agua.merge(total, on="index")
#normaliza pela quantidade de domicíliios na análise
agua['ibge_domicilio01_v012'] = (agua['ibge_domicilio01_v012']/agua['total'])*100

[{'descrição': 'Domicílios particulares permanentes com abastecimento de água '
               'da rede geral',
  'suporte espacial': 'setor_censitario',
  'tabela': 'ibge_domicilio01',
  'variável': 'ibge_domicilio01_v012'}]


In [4]:
visMapaGJson(agua, 
        variavel='ibge_domicilio01_v012', 
        descricao='Domicílios particulares permanentes com abastecimento de água da rede geral',
        width='90%')

As condições domiciliares, avaliadas a partir de indicadores e variáveis de acesso a serviços básicos como água encanada e instalação sanitária, verificamos que a absoluta maioria dos domicílios da região tem abastecimento por água encanada da rede geral. As localidades com menor percentual obtém taxas por volta de 80% e merecem atenção. 

## Infraestrutura sanitária:
- 15 Domicílios particulares permanentes com banheiro de uso exclusivo dos moradores ou sanitário
- 16 Domicílios particulares permanentes com banheiro de uso exclusivo dos moradores ou sanitário e esgotamento sanitário via rede geral de esgoto ou pluvial
- 22 Domicílios particulares permanentes sem banheiro de uso exclusivo dos moradores e nem sanitário
- 23 Domicílios particulares permanentes com banheiro de uso exclusivo dos moradores
- 33 Domicílios particulares permanentes sem banheiro de uso exclusivo dos moradores


In [5]:
banheiro, m = obterTema(indexes='15, 16, 22, 23, 33')
pprint.pprint(m) #imprime as variáveis presentes na informação

#obtendo dados apenas para a área de estudo
banheiro = juncaoEspacial(banheiro, centro, tipo='inner', predicado='intersects')
banheiro = limparDados(banheiro) #limpeza de valores não definidos

banheiro = banheiro.merge(total, on="index")


[{'descrição': 'Domicílios particulares permanentes com banheiro de uso '
               'exclusivo dos moradores ou sanitário',
  'suporte espacial': 'setor_censitario',
  'tabela': 'ibge_domicilio01',
  'variável': 'ibge_domicilio01_v016'},
 {'descrição': 'Domicílios particulares permanentes com banheiro de uso '
               'exclusivo dos moradores ou sanitário e esgotamento sanitário '
               'via rede geral de esgoto ou pluvial',
  'suporte espacial': 'setor_censitario',
  'tabela': 'ibge_domicilio01',
  'variável': 'ibge_domicilio01_v017'},
 {'descrição': 'Domicílios particulares permanentes sem banheiro de uso '
               'exclusivo dos moradores e nem sanitário',
  'suporte espacial': 'setor_censitario',
  'tabela': 'ibge_domicilio01',
  'variável': 'ibge_domicilio01_v023'},
 {'descrição': 'Domicílios particulares permanentes com banheiro de uso '
               'exclusivo dos moradores',
  'suporte espacial': 'setor_censitario',
  'tabela': 'ibge_domicilio01',


In [6]:
#normaliza pela quantidade de domicíliios na análise
banheiro['ibge_domicilio01_v016'] = (banheiro['ibge_domicilio01_v016']/banheiro['total'])*100
banheiro['ibge_domicilio01_v017'] = (banheiro['ibge_domicilio01_v017']/banheiro['total'])*100
banheiro['ibge_domicilio01_v023'] = (banheiro['ibge_domicilio01_v023']/banheiro['total'])*100
banheiro['ibge_domicilio01_v024'] = (banheiro['ibge_domicilio01_v024']/banheiro['total'])*100
banheiro['ibge_domicilio01_v034'] = (banheiro['ibge_domicilio01_v034']/banheiro['total'])*100

In [10]:
visMapaDados(banheiro, m, width='45%', MAPA_ZOOM=14)


A maior parte dos domicílios possuem banheiro, de uso exclusivo dos moradores. Entretanto, existem alguns setores com uma taxa de até 37% sem banheiro (em amarelo) e ainda com baixa cobertura de esgotamento sanitário.

## Infraestrutura de coleta de lixo:
- 35 Domicílios particulares permanentes com lixo coletado por serviço de limpeza


In [11]:
lixo, m = obterTema(indexes='35')
pprint.pprint(m) #imprime as variáveis presentes na informação

#obtendo dados apenas para a área de estudo
lixo = juncaoEspacial(lixo, centro, tipo='inner', predicado='intersects')
lixo = limparDados(lixo) #limpeza de valores não definidos

lixo = lixo.merge(total, on="index")


[{'descrição': 'Domicílios particulares permanentes com lixo coletado por '
               'serviço de limpeza',
  'suporte espacial': 'setor_censitario',
  'tabela': 'ibge_domicilio01',
  'variável': 'ibge_domicilio01_v036'}]


In [14]:
lixo['ibge_domicilio01_v036'] = (lixo['ibge_domicilio01_v036']/lixo['total'])*100
visMapaGJson(lixo, 
        variavel='ibge_domicilio01_v036', 
        descricao='Domicílios particulares permanentes com lixo coletado por serviço de limpeza',
        width='90%')


Observa-se que as áreas ao redor das avenidas possuem cobertura de lixo coletado entre 88 e 100%, entretando existem área mais próximas à Liberdade onde esse índice chega a cair para 59%

## Infraestrutura de energia:
- 43 Domicílios particulares permanentes com energia elétrica de companhia distribuidora

In [15]:
energia, m = obterTema(indexes='43')
pprint.pprint(m) #imprime as variáveis presentes na informação

#obtendo dados apenas para a área de estudo
energia = juncaoEspacial(energia, centro, tipo='inner', predicado='intersects')
energia = limparDados(energia) #limpeza de valores não definidos

energia = energia.merge(total, on="index")


[{'descrição': 'Domicílios particulares permanentes com energia elétrica de '
               'companhia distribuidora',
  'suporte espacial': 'setor_censitario',
  'tabela': 'ibge_domicilio01',
  'variável': 'ibge_domicilio01_v044'}]


In [16]:
energia['ibge_domicilio01_v044'] = (energia['ibge_domicilio01_v044']/energia['total'])*100
visMapaGJson(energia, 
        variavel='ibge_domicilio01_v044', 
        descricao='Domicílios particulares permanentes com energia elétrica de companhia distribuidora',
        width='90%')


Chama a atenção as localidades onde a cobertura de energia elétrica pela distribuidora é inferior à 98% visto que se trata de uma área de grande urbanização da cidade.

## Índice de Infraestrutura (IPEA IVS UDH)

As dimensões do Índice de Vulnerabilidade Social (IVS) permitem a identificação de regiões do espaço urbano onde há sobreposição de condições que podem ser associadas à vulnerabilidade social. Deste modo, esses índices se constituem ferramentas úteis para o planejamento de políticas públicas voltadas para lidar melhor com as fragilidades e vulnerabilidades que se apresenta nesses territórios. 

A dimensão infraestrutura urbana (IVS-I) é obtido através da média ponderada como: Percentual da população que vive em domicílios urbanos sem o serviço de coleta de lixo (peso: 0,300); Percentual de pessoas em domicílios com abastecimento de água e esgotamento sanitário inadequados (peso: 0,300); Percentual de pessoas em domicílios vulneráveis à pobreza e que gastam mais de uma hora até o trabalho no total de pessoas ocupadas, vulneráveis e que retornam diariamente do trabalho (peso: 0,400). 

Analisando o IVS-I para a região em estudo verificamos que a mesmo se encontra no intervalo entre 0,35 e 0,42, pelo pode ser constatado que do ponto de vista da infraestrutura urbana a área de atuação do Programa se encontra na faixa de média a alta vulnerabilidade.

In [19]:
#obtém apenas as geometrias dos setores, para plotar por traz das geometrias de UDM
setor_censitario = obterCamada('setor_censitario', simples=True)

#obtém as geometrias e todos os indicadores do udm
udm, meta_udm = obterIndicador(2669)
pprint.pprint(meta_udm)

#restringindo a área de anaálise para o centro da cidade:
setores_centro = juncaoEspacial(setor_censitario, centro, tipo='inner', predicado='intersects')

#restringindo a área de anaálise para o centro da cidade:
udm_centro = juncaoEspacial(udm, centro, tipo='inner', predicado='intersects')


[{'descrição': 'Índice da dimensão Infraestrutura Urbana, é um dos 3 índices '
               'que compõem o IVS. É obtido através da média ponderada de '
               'índices normalizados construídos a partir dos indicadores que '
               'compõem esta dimensão, a saber: 1) Percentual da população que '
               'vive em domicílios urbanos sem o serviço de coleta de lixo '
               '(peso: 0,300); 2) Percentual de pessoas em domicílios com '
               'abastecimento de água e esgotamento sanitário inadequados '
               '(peso: 0,300); 3) Percentual de pessoas em domicílios '
               'vulneráveis à pobreza e que gastam mais de uma hora até o '
               'trabalho no total de pessoas ocupadas, vulneráveis e que '
               'retornam diariamente do trabalho (peso: 0,400).',
  'suporte espacial': 'ivs_ipea_sao_luis',
  'tabela': 'atlas_vis_udh',
  'variável': 'atlas_vis_udh_ivs_infraestrutura_urbana'}]


In [20]:
m = visMultiMapa(width='80%', MAPA_ZOOM=14)
m = visMultiMapa(m, tipo='layer', dado=setores_centro)
m = visMultiMapa(m, tipo='choro', 
                    dado=udm_centro,                         
                    variavel='atlas_vis_udh_ivs_infraestrutura_urbana',
                    alias='Infraestrutura Urbana')                     
m